In [1]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import time
from features import extract_features, calc_hog_features, calc_bin_spatial_features
from skimage.util.shape import view_as_windows
%matplotlib inline

In [2]:
dist_pickle = pickle.load( open('Car_NoCar_LinearSVC.p', 'r' ) )
svc = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orientations"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
spatial_size = dist_pickle["color_spatial_size"]
h_colorspace = dist_pickle["h_colorspace"]
s_colorspace = dist_pickle["s_colorspace"]
#hog_channel = dist_pickle["hog_channel"]
hog_channel = 'ALL'

In [38]:
#we will be using the same indexing of windows for all images. The data changes, but our indices will not
#Pre make this list. Consumes memory, but saves time
def get_hog_features_(hog_data, patch_size=64, orient=9, channels=3, pix_per_cell=8, cell_per_block=2, cells_per_step=4):
    ncells = (patch_size - 8)//8 + 1
    nblocks = (ncells - cell_per_block)//1 + 1
    window_shape = (channels, nblocks, nblocks, cell_per_block, cell_per_block, orient)
    step = cells_per_step
    feature_vector_len = (channels * nblocks**2 * cell_per_block**2 * orient)
    hog_windows = view_as_windows(hog_data, window_shape, step)
    hog_feature_array = hog_windows.reshape(-1, feature_vector_len)
    print("Hog feature array shape: ", hog_feature_array.shape)
    return hog_feature_array

#Input is an image scaled by 0.5
def get_color_spatial_features_(color_data, patch_size=32, channels=3, pixels_per_step=16):
    #1 cell = 8 pixels for full scale image. Here we use 0.5 scale.
    window_shape = (patch_size, patch_size, channels)
    feature_vector_len = (patch_size**2 * channels)
    step = pixels_per_step
    color_spatial_windows = view_as_windows(color_data, window_shape, step)
    color_spatial_feature_array = color_spatial_windows.reshape(-1, feature_vector_len)
    print("Color Spatial array shape: ", color_spatial_feature_array.shape)
    return color_spatial_feature_array    

def get_feature_array(img_roi, hog_channel, h_cspace, s_space, patch_size=64, orient=9,
                      pix_per_cell=8, cell_per_block=2, cells_per_step=4, pixels_per_step=16):
    hog_data = calc_hog_features(img_roi, orient, pix_per_cell, cell_per_block, 
                                 channel=hog_channel, cspace=h_cspace, vis=False, feature_vec=False)
    #hog_data = np.array(hog_data)
    print("Hog data shape", hog_data.shape)
    hog_feature_array = get_hog_features_(hog_data, patch_size, orient, 3, pix_per_cell, cell_per_block, cells_per_step)
    
    
    color_data = calc_bin_spatial_features(img_roi, cspace=s_space, scale=2, feature_vec=False)
    color_spatial_feature_array = get_color_spatial_features_(color_data, patch_size=32, channels=3, pixels_per_step=pixels_per_step)
    
    feature_array = np.hstack((hog_feature_array, color_spatial_feature_array))
    print("Entire feature Array ", feature_array.shape)
    return feature_array

In [39]:
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, 
              spatial_size, cells_per_step, hog_channel='ALL', show_windows=True):
    
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255
    
    img_roi = img[ystart:ystop,:,:]
    #ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = img_roi.shape
        img_roi = cv2.resize(img_roi, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
    x_size = img_roi.shape[1]
    y_size = img_roi.shape[0]
    print(x_size, y_size)
    fa = get_feature_array(img_roi, hog_channel, h_cspace=h_colorspace, s_space=s_colorspace)
    print(fa.shape)
    return fa
    # Define blocks and steps as above
    #Number of Cells
    nxcells = x_size // pix_per_cell
    nycells = y_size // pix_per_cell
    #Number of Blocks
    nxblocks = (nxcells - cell_per_block) + 1 ##(W-F)/S + 1, W=ncells, F=cell_per_block, S=Stride=1
    nyblocks = (nycells - cell_per_block) + 1 
    #nxblocks = (x_size // pix_per_cell)-1  
    #nyblocks = (y_size // pix_per_cell)-1 
    nfeat_per_block = orient * (cell_per_block**2)
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = ((window // pix_per_cell) - cell_per_block) + 1
    
    #We have 8 cells per 64 pixels
    #Each cell step (8 pixels), refers to a move of 1/8 or overlap of 7/8=87.5% overlap
    #A step of 2 cells, means we slide windows by 16 pixels, overlap of 75%.
    #cells_per_step = 2  # Instead of overlap, define how many cells to step
    #Number of steps we can take over HoG array
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog_array = extract_features(img_roi, h_cspace=h_colorspace, s_cspace=s_colorspace, orient=orient, 
                           pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                           hog_channel=hog_channel, feature_vec=False, vis=False)
    #The hog_array we get has a shape 3XnyblocksXnxblocksXcell_per_blockXcell_per_blockXorientations
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_features = hog_array[:, ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            
            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell
            # Extract the image patch
            subimg = cv2.resize(img_roi[ytop:ytop+window, xleft:xleft+window, :], (64,64))
            
            # Get color features
            spatial_features = calc_bin_spatial_features(subimg, s_colorspace)
            #hist_features = color_hist(subimg, nbins=hist_bins)
            
            if show_windows:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                color = np.random.randint(0, 256, (3,))
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart), 
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),color,6) 
            # Scale features and make a prediction
            test_features = X_scaler.transform(np.hstack((hog_features, spatial_features)).reshape(1, -1))
            #test_features = X_scaler.transform(hog_features.reshape(1,-1))
            #test_features = X_scaler.transform(np.hstack((shape_feat, hist_feat)).reshape(1, -1))    
            test_prediction = svc.predict(test_features)
            
            if test_prediction == 1:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                #color = np.random.randint(0, 256, (3,))
                cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),
                              (xbox_left+win_draw, ytop_draw+win_draw+ystart),(0,0,255),6) 
             
    return draw_img

In [40]:
img = cv2.imread('test_images/test8.jpg') 
print(img.shape)

(738L, 1280L, 3L)


In [45]:
def find_cars_1(img):
    window = 64
    ystart = 400
    ystop = ystart + (window*3)
    scale = 1 #aSteps 5
    t1 = time.time()
    out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell,
                        cell_per_block, spatial_size, cells_per_step=5, hog_channel=hog_channel, show_windows=False)
    return out_img
#print("Time taken", time.time()-t1)
#plt.imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))

In [42]:
7*7*2*2*9*3

5292

In [43]:
fa = find_cars_1(img)

(128L, 128L)
('Hog data shape', (3L, 15L, 15L, 2L, 2L, 9L))
('Hog feature array shape: ', (9L, 5292L))
('Color Spatial array shape: ', (9L, 3072L))
('Entire feature Array ', (9L, 8364L))
(9L, 8364L)


In [44]:
svc.predict(fa)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [72]:
def find_cars_2(img):
    ystart = 400 + (window)
    ystop = ystart + (window*6)
    scale = 1.8 #steps 4

    t1 = time.time()
    out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell,
                        cell_per_block, spatial_size, cells_per_step=6, hog_channel=hog_channel, show_windows=False)
    #print("Time taken", time.time()-t1)
    return out_img
#plt.imshow(cv2.cvtColor(out_img, cv2.COLOR_BGR2RGB))
#plt.show()

In [73]:
def combine(img1, img2):
    return (cv2.addWeighted(img1, 0.5, img2, 0.5, 0))

In [74]:
cap = cv2.VideoCapture('project_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
recorder = cv2.VideoWriter('out.mp4', fourcc, 30, (1280, 720))
t1 = time.time()
cnt = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        cnt+=1;
        #print("cnt", cnt)
        if cnt == 300:
            break
        if cnt%10 == 0:
            print('Time taken ', time.time()-t1)
        f1 = find_cars_1(frame)
        f2 = find_cars_2(frame)
        #print(f1.shape)
        #print(f2.shape)
        img_boxes = combine(find_cars_1(frame), find_cars_2(frame))
        recorder.write(img_boxes)
    else:
        break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
print('Time taken ', time.time()-t1)
cap.release()
recorder.release()
cv2.destroyAllWindows()

(64L, 64L)
('Hog data shape', (3L, 7L, 7L, 2L, 2L, 9L))


ValueError: `window_shape` is too large